In [1]:
# Pakete importieren
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from treeinterpreter import treeinterpreter
from sklearn.tree import DecisionTreeClassifier

# Transformierte Daten laden
with open('../output/titanic/datasets_transformed.pkl', 'rb') as handle:
    datasets_transformed = pickle.load(handle)

# Transformer laden
with open('../output/titanic/transformer_pipeline.pkl', 'rb') as handle:
    transformer_pipeline = pickle.load(handle)

# Datensets laden
with open('../output/titanic/datasets.pkl', 'rb') as handle:
    datasets = pickle.load(handle)

# Full Pipeline erstellen
full_pipeline = Pipeline(steps=[
    ('transformers', transformer_pipeline),
    ('predictor', DecisionTreeClassifier(random_state=0))
])

# Parameter optimieren

In diesem Abschnitt wird gezeigt wie man mit der Grid-Search Methode optimale Parametereinstellungen finden kann.

## Grid Search anwenden

Für jeden beliebigen Schritt in der Pipeline können Wertebereiche für die Parameter angegeben werden. Für das Anwendungsbeispiel werden folgende Wertebereiche gesetzt:
* der Faktor, der bei der IQR-Methode zur Ausreißererkennung verwendet wird, mit einem Wertebereich von [1.0, 1.5, 2.0, 3.0]
* der min_samples_split Parameter, der beim DecisionTreeClassifier-Algorithmus bestimmt wie viele Daten notwendig sind, um eine weitere Verzweigung zu erstellen, mit einem Wertebereich von [2, 3, 4, 5, 6]

Die Bezeichnung der Parameter folgt einer Regel: Vor die eigentliche Parameterbezeichnung wie z.B. "factor" werden die Pipeline und Transformer-Namen mit doppeltem Unterstrich getrennt gestellt.

Der Aufbau der Pipeline:

```{figure} ../images/pipelineStructure.png
---
height: 250px
align: center
name: fig-pipelineStructure
---
```

Die Vollständige Bezeichnung für den Parameter "factor" lautet: "transformers__num__outlier_remover__factor"

Der Parameter param_grid stellt der GridSearchCV Klasse ein Dictionary bereit. Die Keys entsprechen den Bezeichnungen und die Values einer Liste von Werten, die getestet werden sollen.

In [2]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'transformers__num__outlier_remover__factor': [1.0, 1.5, 2.0, 3.0],
    'predictor__min_samples_split' : [2,3,4,5,6]
}

grid_search = GridSearchCV(full_pipeline, param_grid, cv=10)

Die fit()-Methode testet alle Kombinationsmöglichkeiten und speichert die beste Parameterkombination im Attribut best_params_. 

In [3]:
grid_search.fit(datasets['X_train'], datasets['y_train'])
grid_search.best_params_

{'predictor__min_samples_split': 6,
 'transformers__num__outlier_remover__factor': 3.0}

Über das Attribut best_score_ erhält man die Accuracy-Score der besten Parametereinstellung.

In [4]:
grid_search.best_score_

0.7682828282828283

Das Ergebnis auf den Validierungsdaten lautet:

In [5]:
grid_search.score(datasets['X_val'], datasets['y_val'])

0.7624309392265194

Im Vergleich zur vorherigen Einstellung, der Baseline, hat sich das Ergebnis um etwa 1% verbessert.

Die Grid-Search Methode hat für jede Kombinationsmöglichkeit eine Pipeline erstellt. Das Ergebnis zeigt, dass die Pipeline mit den besten Ergebnissen den min_samples_split-Parameter auf 6 und den factor-Parameter auf 3.0 gesetzt hat.

```{figure} ../images/gridsearchDct.png
---
height: 250px
align: center
name: fig-gridsearchDct
---
```

Was fällt bei dem Ergebnis der Grid-Search Methode auf?

```{dropdown} Die besten Parametereinstellungen sind jeweils die kleinsten Werte der vorgegebenen Wertemengen
<font color='red'>Falsch.</font>
```
```{dropdown} Die besten Parametereinstellungen liegen im mittleren Bereich der vorgegebenen Wertemengen
<font color='red'>Falsch.</font>
```
```{dropdown} Die besten Parametereinstellungen sind jeweils die größten Werte der vorgegebenen Wertemengen
<font color='green'>Richtig.</font>
```

Welche Werte sollten in den zu testenden Wertemengen hinzugefügt werden?
```{dropdown} Größere Werte.
<font color='green'>Richtig.</font>
```
```{dropdown} Keine Werte.
<font color='red'>Falsch.</font>
```
```{dropdown} Kleinere Werte.
<font color='red'>Falsch.</font>
```

Es wurde gezeigt, wie die Grid-Search Methode angewendet wird um Parametereinstellungen zu optimieren. Eine weitere Optimierung könnte mit dem Testen einer erweiterten Wertemenge oder/und weiteren Parametern erreicht werden. Um Wiederholung zu vermeiden, brechen wir an dieser Stelle ab und geben uns zunächst mit diesem, bereits besseren, Ergebnis zufrieden. 

## Beste Pipeline auswählen

Die bisherige Pipeline wird nun mit der besten Pipeline aus Grid-Search ersetzt.

In [8]:
best_pipeline_gridsearch = grid_search.best_estimator_

Zum Abschluss: Validieren der Pipeline mit dem Validierungsset. Das Ergebnis liefert wie erwartet einen Accuracy-Score von 0.762

In [9]:
best_pipeline_gridsearch.score(datasets['X_val'], datasets['y_val'])

0.7624309392265194

Aufbauend auf den ermittelten Parametereinstellungen in diesem Abschnitt, werden im nächsten Abschnitt die Merkmalsoptimierung besprochen. 